In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout = nn.Dropout(0.25)
        self.full1 = nn.Linear(32 * 12 * 12, 128)  # 🔹 vuelve a "full1"
        self.full2 = nn.Linear(128, 10)            # 🔹 vuelve a "full2"

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.full1(x))
        x = self.dropout(x)
        x = self.full2(x)
        return x


In [7]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
model.load_state_dict(torch.load("modelo_mnist.pth", map_location=device))
model.eval()

print("✅ Modelo cargado en", device)


✅ Modelo cargado en cpu


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, download=True, transform=transform),
    batch_size=1000,
    shuffle=False
)

# Evaluación rápida
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

print(f"Precisión en test: {100. * correct / len(test_loader.dataset):.2f}%")


Precisión en test: 98.95%
